# COGS 189: Final Project
### Members: Ehson Pirouzian, ... write your own

This is our Winter 2022 Final Project for COGS 189
The purpose is a machine learning project that relates to motor sensory data from BCI competition IV dataset 2a

### Hypothesis

Is it easiest to classify data left hand sensory data against right hand, foot, or tounge sensory data?

### Code

We will use matlab and python for how we test our hypothesis


#### Importing the data

In [104]:
import os
import numpy as np
import pandas as pd
from pprint import pprint
import scipy.io as sio
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

In [103]:

def get_data(subject,training,PATH):
	NO_channels = 22
	NO_tests = 6*48 	
	Window_Length = 7*250 

	class_return = np.zeros(NO_tests)
	data_return = np.zeros((NO_tests,NO_channels,Window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(PATH+'A0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(PATH+'A0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2=[a_data1[0,0]]
		a_data3=a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_fs 		= a_data3[3]
		a_classes 	= a_data3[4]
		a_artifacts = a_data3[5]
		a_gender 	= a_data3[6]
		a_age 		= a_data3[7]
		for trial in range(0,a_trial.size):
			if(a_artifacts[trial]==0):
				data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:22])
				class_return[NO_valid_trial] = int(a_y[trial])
				NO_valid_trial +=1


	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]
	

# Now we will import each of the 9 training files
df1, classes1 = get_data("1",True,"./dataset2a/")
df2, classes2 = get_data("2",True,"./dataset2a/")
df3, classes3 = get_data("3",True,"./dataset2a/")
df4, classes4 = get_data("4",True,"./dataset2a/")
df5, classes5 = get_data("5",True,"./dataset2a/")
df6, classes6 = get_data("6",True,"./dataset2a/")
df7, classes7 = get_data("7",True,"./dataset2a/")
df8, classes8 = get_data("8",True,"./dataset2a/")
df9, classes9 = get_data("9",True,"./dataset2a/")


In [105]:
# 4 classes, #some number of tests, 3 channels, 1750 datapoints each
# this will become 4 different dataframes for each class with 3 channels and #tests for each channel * datapoints

df1 = df1[:,(7,9,11),:]
df2 = df2[:,(7,9,11),:]
df3 = df3[:,(7,9,11),:]
df4 = df4[:,(7,9,11),:]
df5 = df5[:,(7,9,11),:]
df6 = df6[:,(7,9,11),:]
df7 = df7[:,(7,9,11),:]
df8 = df8[:,(7,9,11),:]
df9 = df9[:,(7,9,11),:]



In [151]:
raw = mne.io.read_raw_gdf('./BCICIV_2a_gdf/A01T.gdf', preload=True)
# print(raw['data'][1])
eegbci.standardize(raw)  # set channel names
print(raw.info)
montage = mne.channels.make_standard_montage('biosemi256')
raw.set_montage(montage, on_missing='ignore')

raw.rename_channels(lambda x: x.strip('.'))
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')


Extracting EDF parameters from /Users/ehson/Documents/Classes/COGS189/Cogs-189-Project/BCICIV_2a_gdf/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/Users/ehson/miniconda3/envs/cogs/lib/python3.9/site-packages/mne/io/edf/edf.py:1131: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/Users/ehson/miniconda3/envs/cogs/lib/python3.9/contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


<Info | 7 non-empty values
 bads: []
 ch_names: EEG-Fz, EEG-0, EEG-1, EEG-2, EEG-3, EEG-4, EEG-5, EEG-C3, EEG-6, ...
 chs: 25 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 100.0 Hz
 meas_date: 2005-01-17 12:00:00 UTC
 nchan: 25
 projs: []
 sfreq: 250.0 Hz
>
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 sec)



<RawGDF | A01T.gdf, 25 x 672528 (2690.1 s), ~128.3 MB, data loaded>

In [158]:
mne.events_from_annotations(raw)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


(array([[     0,      0,      5],
        [     0,      0,      3],
        [ 29683,      0,      5],
        ...,
        [670550,      0,      6],
        [670550,      0,      1],
        [671050,      0,      7]]),
 {'1023': 1,
  '1072': 2,
  '276': 3,
  '277': 4,
  '32766': 5,
  '768': 6,
  '769': 7,
  '770': 8,
  '771': 9,
  '772': 10})

In [162]:
def filterEventsLeft_Foot(events):
    finalArray = []
    for event in events:
        if event[2] == 8 | event[2] == 9:
            finalArray.append(event)
    return finalArray

events, _ = events_from_annotations(raw)
events = filterEventsLeft_Foot(events)

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

event_id = dict(hands=8, feet=9)
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
[[     0      0      5]
 [     0      0      3]
 [ 29683      0      5]
 ...
 [670550      0      6]
 [670550      0      1]
 [671050      0      7]]
[array([94371,     0,     9]), array([104277,      0,      9]), array([110256,      0,      9]), array([128170,      0,      9]), array([138329,      0,      9]), array([158713,      0,      9]), array([160756,      0,      9]), array([162820,      0,      9]), array([166812,      0,      9]), array([174759,      0,      9]), array([178868,      0,      9]), array([184816,      0,      9]), array([199195,      0,      9]), array([203162,      0,      9]), array([210892,      0,      9]), array([212864,      0,      9]), array([216775,      0,      9]), array([227071,      0,      9]), array([241182,      0,      9]), array([247187,      0,      9]), array([249258,      0,      9]), array([253457,      0,      9]), array([271594,      

Here I will explain our reasoning for the above code






As a citation of sources we used this data set as a guide for formatting our data

1. https://github.com/MultiScale-BCI/IV-2a/blob/master/  by "Michael Hersche and Tino Rellstab"

In [112]:
#print(df1.shape)

# classes1
# a = sio.loadmat("./matFiles/A01T.mat")
# data = (a['EEG'][0][0][15])

# print(data.shape)
# print(df1.shape)


# band pass
df1 = mne.filter.filter_data(df1,sfreq=250, l_freq =7., h_freq=30., fir_design='firwin')

print(df1.shape)
# epoch the data 

# epochs = Epochs(df1, classes1, [2.0, 3.0], 3, 6, proj=True, baseline=None, preload=True)



Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 sec)

(273, 3, 1750)


ValueError: The first argument to `Epochs` must be an instance of mne.io.BaseRaw